# Testes de Desempenho

## K-Means

Vamos agora realizar testes de ganho de velocidade de execução, comparando o desempenho do K-Means rodando na CPU com o do K-means rodando na GPU.

Desta vez, iremos utilizar datasets bem maiores e, portanto, nada triviais — como era o caso do [*Iris* Data Set](https://archive.ics.uci.edu/ml/datasets/Iris) que foi usado anteriormente apenas como uma prova de conceito e teste de corretude.

A ideia é testar se os ganhos de desempenho ao utilizarmos uma versão paralelizada em GPU diminuem, estagnam ou aumentam junto com o aumento de instâncias ou dimensionalidade do dataset.

Também será testado se houve diferença de precisção de cada classificação. Isso é realizado usando a função `getClassificationHits()`, explicada mais a fundo nos cadernos Jupyter `kMeansCPU.ipynb` e `kMeansGPU.ipynb`.

### Código Comum

In [2]:
import kMeans as km
import pandas as pd

import time
import os

import importlib
importlib.reload(km)

# Testing imports
print(km.kMeansCPU)
print(km.kMeansGPU)
print(km.getClassificationHits)

# Se verdadeiro, os testes incluirão a contagem de acertos dos resultados dos algoritmos. Isso pode demorar MUITO (>5h por execução no dataset 5)!
TEST_CORRECTEDNESS = True

# Valor efetivamente infinito para um float, para ser usado como valor inicial na variável "fastestExecTime"
FLOAT_MAX = float('inf')

# Configurando Numba para não reportar erros de baixa ocupação dos streaming multiprocessors (SMs) da GPU
# Não suprimir estes erros gera um overhead bem considerável, ocasionalmente, em algumas execuções do K-Means GPU
%set_env NUMBA_CUDA_LOW_OCCUPANCY_WARNINGS 0

# Função para rodar os testes
def runTests(mode:str='GPU', runs:int=10, countHits:bool=True):
    '''Essa função depende de diversas variáveis declaradas anteriormente. Portanto, é inútil fora deste caderno Jupyter!'''

    mode = mode.upper()
    if mode == 'CPU': kMeans = km.kMeansCPU
    elif mode == 'GPU': kMeans = km.kMeansGPU
    else: raise ValueError('Unknown mode!')

    totalExecTime = 0.0
    slowestExecTime = -1.0
    fastestExecTime = FLOAT_MAX

    totalHits = 0
    totalHitsTime = 0.0

    for rep in range(1, runs + 1):
        startTime = time.perf_counter()
        results = kMeans(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
        elapsedTime = time.perf_counter() - startTime
        if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
        if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
        totalExecTime += elapsedTime
        print(f'Execution K-Means {mode} run #{rep}: {elapsedTime}; curr avg: {totalExecTime / rep}; ', end='')

        if countHits:
            # Verificando acertos
            # Converting from numpy arrays to panda's dataframes, if needed
            if results.__class__.__name__ != pd.DataFrame.__class__.__name__: results = pd.DataFrame(results)
            startTime = time.perf_counter()
            hits, _, _  = km.getClassificationHits(results, dataset, classColumnName, classes, debug=DEBUG)
            elapsedTime = time.perf_counter() - startTime
            totalHits += hits
            totalHitsTime += elapsedTime
            print(f'Hits: {hits} (done in {elapsedTime:.4f}); curr avg hits: {totalHits / rep}', end='')

        print('\n', end='')

    print(f' \nAvg exec K-Means {mode}: {totalExecTime / runs}')
    print(f'Max exec K-Means {mode}: {slowestExecTime}')
    print(f'Min exec time K-Means {mode}: {fastestExecTime}')

    if countHits:
        print(f' \nAverage hits: {totalHits / runs}')
        print(f'Avg exec K-Means {mode} classificationHits(): {totalHitsTime / runs}')

<function kMeansCPU at 0x71994c0e82c0>
<function kMeansGPU at 0x71994c0e9760>
<function getClassificationHits at 0x719947f1b6a0>
env: NUMBA_CUDA_LOW_OCCUPANCY_WARNINGS=0


### Dataset 1 (N > 1.000, D = 7, K = 2) — Rice (Cammeo and Osmancik)

Foi utilizado aqui o Dataset **[Rice (Cammeo and Osmancik)](https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik)**, que reúne dados expressando características morfológicas de grãos de arroz de duas espécies, extraídas a partir de fotos destes. Temos **7 variáveis (D = 7)** e **3.810 instâncias**.

Esse dataset também contém informações de classe, definindo qual a espécie real do grão de arroz: **Cammeo** ou **Osmancik**. Portanto, haverão **2 grupos de dados (K = 2)**.

Esse conjunto de dados está presente no arquivo `Rice_Cammeo_Osmancik.arff` dentro do arquivo `rice+cammeo+and+osmancik.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/545/rice+cammeo+and+osmancik.zip)).

#### Código

In [ ]:
# Novas variáveis globais
K = 2
MAX_ITERATIONS = 60
PLOT_RESULTS = False
DEBUG = False

COMMENT_CHAR = '%'
ALTERNATIVE_COMMENT_CHARS = ['@']

datasetFilePath = './Rice_Cammeo_Osmancik.csv'

# Processando o aqruivo .arff file e convertendo para um arquivo .csv válido (com linhas comentadas)
if not os.path.exists(datasetFilePath):
    with \
        open('./rice+cammeo+and+osmancik/Rice_Cammeo_Osmancik.arff', 'r') as file,\
        open(datasetFilePath, 'w') as fileNew:

        for line in file:
            if line[0] in ALTERNATIVE_COMMENT_CHARS:
                fileNew.write(COMMENT_CHAR + ' ' + line[1:])
            else:
                fileNew.write(line)

columnNames = ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent', 'Class']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep=',', skip_blank_lines=True, comment=COMMENT_CHAR)

datasetTreated = dataset.drop(columns=['Class'])
print(datasetTreated)

classColumnName = 'Class'
classes = dataset[classColumnName].unique()

print(f'Classes (from column "{classColumnName}"): {classes}')

In [ ]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((datasetTreated - datasetTreated.min()) / (datasetTreated.max() - datasetTreated.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

In [ ]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

# runTests('CPU', 100, TEST_CORRECTEDNESS)

In [ ]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

""" NUMBER_OF_RUNS = 10

totalExecTime = 0.0
slowestExecTime = -1.0
fastestExecTime = FLOAT_MAX

totalHits = 0
totalHitsTime = 0.0

for rep in range(1, NUMBER_OF_RUNS + 1):
    startTime = time.time()
    results = km.kMeansGPU(datasetTreated, k=K, maxIter=MAX_ITERATIONS, printIter=False, plotResults=PLOT_RESULTS, debug=DEBUG)
    elapsedTime = time.time() - startTime
    if elapsedTime < fastestExecTime: fastestExecTime = elapsedTime
    if elapsedTime > slowestExecTime: slowestExecTime = elapsedTime
    totalExecTime += elapsedTime
    print(f'Execution K-Means GPU run #{rep}: {elapsedTime}; curr avg: {totalExecTime / rep}; ', end='')

    # Verificando acertos
    # Converting from numpy arrays to panda's dataframes, if needed
    if results.__class__.__name__ != pd.DataFrame.__class__.__name__: results = pd.DataFrame(results)
    startTime = time.time()
    hits, _, _  = km.getClassificationHits(results, dataset, classColumnName, classes, debug=DEBUG)
    elapsedTime = time.time() - startTime
    totalHits += hits
    totalHitsTime += elapsedTime
    print(f'Hits: {hits} (done in {elapsedTime:.4f}); curr avg hits: {totalHits / rep}\n')

print(f' \nAvg exec K-Means GPU: {totalExecTime / NUMBER_OF_RUNS}')
print(f'Max exec K-Means GPU: {slowestExecTime}')
print(f'Min exec time K-Means GPU: {fastestExecTime}')

print(f' \nAverage hits: {totalHits / NUMBER_OF_RUNS}')
print(f'Avg exec K-Means GPU classificationHits(): {totalHitsTime / NUMBER_OF_RUNS}') """

# runTests('GPU', 100, TEST_CORRECTEDNESS)

### Dataset 2 (N > 10.000, D = 8, K = 2) — HTRU2

Foi utilizado aqui o Dataset **[HTRU2 (High Time Resolution Universe 2)](https://archive.ics.uci.edu/dataset/372/htru2)**, que reúne dados a respeito de emissões de sinais de rádio de banda larga obtidos através de leituras feitas com telescópios de rádio. É um dos resultados da busca por pulsares, estrelas de neutrôn que possuem uma rotação rápida e que emitem sinais de rádio banda larga detectáveis do nosso planeta. Temos **8 variáveis (D = 8)** e **17.898 instâncias**.

Esse dataset também contém informações de classe, definindo se a leitura é **positiva** ou **negativa**, a respeito do sinal candidato de fato originar ou não de um pulsar. Portanto, haverão **2 grupos de dados (K = 2)**.

Esse conjunto de dados está presente no arquivo `HTRU_2.csv` dentro do arquivo `HTRU_2.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/372/htru2.zip)).

#### Código

In [ ]:
# Novas variáveis globais
K = 2
MAX_ITERATIONS = 60
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './htru2/HTRU_2.csv'

columnNames = ['mean_IP', 'std_dev_IP', 'exc_kurt_IP', 'skew_IP', 'mean_DM_SNR', 'std_dev_DM_SNR', 'exc_kurt_DM_SNR', 'skew_DM_SNR', 'is_positive']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep=',')

datasetTreated = dataset.drop(columns=['is_positive'])
print(datasetTreated)

classColumnName = 'is_positive'
classes = dataset[classColumnName].unique()

print(f'Classes (from column "{classColumnName}"): {classes}')

In [ ]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((datasetTreated - datasetTreated.min()) / (datasetTreated.max() - datasetTreated.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

In [ ]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

# runTests('CPU', 100, TEST_CORRECTEDNESS)

In [ ]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

# runTests('GPU', 100, TEST_CORRECTEDNESS)

### Dataset 3 (N > 100.000, D = 50, K = 2) — MiniBooNE

Foi utilizado aqui o Dataset **[MiniBooNE Particle Identification](https://archive.ics.uci.edu/dataset/199/miniboone+particle+identification)**, que reúne dados a respeito de partículas detectadas no experimento *MiniBooNE* (*Mini Booster Neutrino Experiment*), conduzido no laboratório americano *Fermilab*. Cada detecção de partícula é descrita por **50 variáveis reais (D = 50)** e há **129.596 instâncias no total**.

As primeiras 36.488 instâncias são detecções de neutrinos do elétron (sinal) e as 93.108 restantes são de neutrinos do múon (ruído de fundo). Assim, as informações de classe desse dataset estão implícitas, expressa pela ordem das instâncias no arquivo. Como temos duas classes, haverão **2 grupos de dados (K = 2)**.

Esse conjunto de dados está presente no arquivo `MiniBooNE_PID.txt` dentro do arquivo `miniboone+particle+identification.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/199/miniboone+particle+identification.zip)).

Foi necessário, neste dataset, realizar um **pré-processamento** para **remoção de outliers**. Originalmente, há 130.064 instâncias no total (36.499 sinal e 93.565 ruído). Porém, existem 468 instâncias (11 sinal e 457 ruído) que são extremos outliers, possuindo o valor -999.0 em todas as 50 variáveis — provavelmente advindos de algum erro de detecção. A presença destes outliers causava a criação de um cluster contendo apenas estes outliers, diminuindo muito o tempo de execução do algoritmo de maneira artificial. Estes outliers tiveram que ser removidos. Note que poderíamos ter solucionado este problema com outra abordagem: aumentar K para 3, criando um cluster novo para conter apenas os outliers. Isso, no entanto, seria mais custoso computacionalmente do que a remoção das instâncias.

#### Código

In [ ]:
# Novas variáveis globais
K = 2
MAX_ITERATIONS = 60
PLOT_RESULTS = False
DEBUG = False

COMMENT_CHAR = '#'

# As primeiras 36.499 instâncias são consideradas um sinal, e o resto como ruído
N_OF_SIGNAL_LINES = 36499

datasetFilePath = './MiniBooNE_PID.csv'

# Processando o aqruivo .txt file e convertendo para um arquivo .csv válido (com a primeira linha comentada, removendo o leading whitespace, e trocando o separador de "  " ou " " para ",")
if not os.path.exists(datasetFilePath):
    with \
        open('./MiniBooNE_PID.txt', 'r') as file,\
        open(datasetFilePath, 'w') as fileNew:

        print('Processing MiniBooNE_PID.txt...\n ')

        # Removendo outliers com -999.0 de valor nas 50 variáveis. Há 468 destas instâncias
        outlierString = '''-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03,-0.999000E+03'''

        index = 1
        signalInstRemoved = 0
        noiseInstRemoved = 0
        for line in file:
            if index != 1:
                lineToWrite = line.strip(' ').replace('  ', ' ').replace(' ', ',')
                if outlierString not in lineToWrite:
                    fileNew.write(lineToWrite)
                else:
                    if index - 1 <= N_OF_SIGNAL_LINES:
                        # print(f'Instance (signal) #{index - 1} removed...')
                        signalInstRemoved += 1
                    else:
                        # print(f'Instance (noise) #{index - 1} removed...')
                        noiseInstRemoved += 1
            # else:
            #     fileNew.write(COMMENT_CHAR + ' ' + line.strip(' ').replace('  ', ' '))
            index += 1

        print(f'Signal outlier instances removed = {signalInstRemoved}')
        print(f'Noise outlier instances removed = {noiseInstRemoved}\n ')

        print(f'Processed dataset saved in {datasetFilePath} with success!\n ')
else:
    print(f'Processed dataset found in {datasetFilePath}. No need for processing!\n ')

columnNames = [f'id_var_{i}' for i in range(1, 50 + 1) ]

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFile, names=columnNames, sep=',', skip_blank_lines=True)

# Gerando coluna de classes
classColumn = pd.DataFrame(['signal' if idx <= 36488 else 'noise' for idx in range(1, len(dataset) + 1)])
# print(classColumn)

classColumnName = 'class'
dataset.insert(len(dataset.columns), classColumnName, classColumn)
del classColumn

datasetTreated = dataset.drop(columns=[classColumnName])
print(datasetTreated)

classes = ['signal', 'noise']
print(f'Classes (from column "{classColumnName}"): {classes}')

In [ ]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((datasetTreated - datasetTreated.min()) / (datasetTreated.max() - datasetTreated.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

In [ ]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

# runTests('CPU', 50, TEST_CORRECTEDNESS)

In [ ]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

# runTests('GPU', 50, TEST_CORRECTEDNESS)

### Dataset 4 (N > 1.000.000, D = 8) — WESAD

Foi utilizado aqui um sub-conjunto dos dados do Dataset **[WESAD (Wearable Stress and Affect Detection)](https://archive.ics.uci.edu/dataset/465/wesad+wearable+stress+and+affect+detection)**, que reúne dados, fisiológicos e de movimento, de diversos sensores presentes em aparelhos *wearables* usados por 15 pacientes diferentes em testes laboratoriais. Um aparelho foi usado no peitoral e outro no pulso dos pacientes.

Esse dataset também contém informações de classe, definindo momentos dos testes como pertencendo à três classificações de emoção do paciente: **referência**, **estresse** ou **diversão**. Portanto, haverão **3 grupos de dados (K = 3)**.

O sub-conjunto de dados utilizado foi: dados obtidos apenas através do **aparelho usado no peito** do paciente, e apenas do **paciente #4**. Utilizando este sub-conjunto, temos **8 variáveis (D = 8)** e **4.588.552 instâncias**, cada uma sendo uma leitura ao longo do tempo do teste laboratorial (leituras realizadas na frequência de 700hz).

Esse sub-conjunto de dados está presente no arquivo `S4/S4_respiban.txt` dentro do arquivo `WESAD.zip` do dataset (também disponível em download direto [neste link](https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download)).

#### Código

In [ ]:
# Novas variáveis globais
K = 3
MAX_ITERATIONS = 60
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './WESAD/S4/S4_respiban.txt'
columnNames = ['index', 'DI', 'ECG', 'EDA', 'EMG', 'TEMP', 'spatialX', 'spatialY', 'spatialZ', 'RESPIRATION', '_ignore_']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFilePath, names=columnNames, sep='\t', index_col=0, skip_blank_lines=True, comment='#')

# datasetTreated = dataset.drop(columns=['DI', '_ignore_'])
# print(datasetTreated)

# classColumnName = 'DI'
# classes = dataset[classColumnName].unique()

# print(f'Classes (from column "{classColumnName}"): {classes}')

# Gerando coluna de classes
classColumn = []
for idx in range(len(dataset)):
    classification = None
    if idx < 1329300: classification = 'base'
    elif idx < 1926400: classification = 'fun'
    elif idx < 2563400: classification = 'base' # Medi 1
    elif idx < 4020100: classification = 'stress'
    else: classification = 'base' # Medi 2

    classColumn.append(classification)
classColumn = pd.DataFrame(classColumn)
print(classColumn)

classColumnName = 'class'
dataset.insert(len(dataset.columns), classColumnName, classColumn)
del classColumn

datasetTreated = dataset.drop(columns=['DI', '_ignore_', classColumnName])
print(datasetTreated)

classes = ['base', 'fun', 'stress']
print(f'Classes (from column "{classColumnName}"): {classes}')

In [ ]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((datasetTreated - datasetTreated.min()) / (datasetTreated.max() - datasetTreated.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

In [ ]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

# runTests('CPU', 5, TEST_CORRECTEDNESS)

In [ ]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

# runTests('GPU', 5, TEST_CORRECTEDNESS)

#### Resultados

> Resultados completos disponíveis no arquivo `code/examples-and-tests/speedupTestsRawResults.txt`

| |Tempo médio (50 execuções)|Speedup Médio|
|-|-|-|
|K-Means CPU|~129,81s|-|
|K-Means GPU|~27,87s|~4,65x|

### Dataset 5 (N > 10.000.000, D = 3, K = 7) — HHAR

Foi utilizado aqui um sub-conjunto dos dados do Dataset **[Heterogeneity Human Activity Recognition (HHAR)](https://archive.ics.uci.edu/dataset/344/heterogeneity+activity+recognition)**, que reúne dados de movimento do giroscópio e acelerômetro presentes em aparelhos celulares (*smartphones*) e relógios (*smartwatches*) usados por 9 usuários diferentes ao realizar diversas atividades físicas diferentes ou estando em repouso.

Esse dataset também contém informações de classe, definindo momentos dos testes como pertencendo a uma de seis atividades realizadas: **ciclismo**, **repouso (sentado)**, **repouso (em pé)**, **andar**, **subir escadas** e **descer escadas**. Além disto, há uma sétima "atividade", **nula**, que representa espaços do teste onde não foi realizada nenhuma atividade. Portanto, haverão **7 grupos de dados (K = 7)**.

O sub-conjunto de dados utilizado foi: dados obtidos apenas através do **giroscópio do smartphone** do usuário. Utilizando este sub-conjunto, temos **3 variáveis (D = 3)** e **13.932.632 instâncias**, cada uma sendo uma leitura ao longo do tempo do experimento.

Esse sub-conjunto de dados está presente no arquivo `Activity recognition exp/Phones_gyroscope.csv` dentro do arquivo `heterogeneity+activity+recognition.zip` do dataset (também disponível em download direto [neste link](https://archive.ics.uci.edu/static/public/344/heterogeneity+activity+recognition.zip)).

#### Código

In [ ]:
# Novas variáveis globais
K = 7
MAX_ITERATIONS = 60
PLOT_RESULTS = False
DEBUG = False

datasetFilePath = './heterogeneity+activity+recognition/Activity recognition exp/Phones_gyroscope.csv'
columnNames = ['index', 'arrival_time', 'creation_Time', 'x', 'y', 'z', 'user', 'model', 'device', 'gt']

# Lendo dataset do arquivo
with open(datasetFilePath, 'r') as datasetFile:
    dataset = pd.read_csv(datasetFile, names=columnNames, header=0, sep=',', index_col=0)

datasetTreated = dataset.drop(columns=['arrival_time', 'creation_Time', 'user', 'model', 'device', 'gt'])
print(datasetTreated)

classColumnName = 'gt'
classes = dataset[classColumnName].unique()

print(f'Classes (from column "{classColumnName}"): {classes}')

In [ ]:
# Normalizando o dataset (normalização min-max), para que todos valores estejam no intervalo [1, 10]
datasetTreated = ((datasetTreated - datasetTreated.min()) / (datasetTreated.max() - datasetTreated.min())) * 9 + 1

print(f'##### Dataset (tratado e normalizado, intervalo [1, 10]) #####\n{datasetTreated}')

In [ ]:

# * ####################################
# * Rodando o K-Means CPU
# * ####################################

runTests('CPU', 1, TEST_CORRECTEDNESS)

In [ ]:

# * ####################################
# * Rodando o K-Means GPU
# * ####################################

runTests('GPU', 2, TEST_CORRECTEDNESS)